An editted version of the demo notebook in https://github.com/matterport/Mask_RCNN

# Install libraries

In [ ]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install kaggle

# Clone Repo

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
os.chdir('/content/drive/MyDrive/Mask_RCNN/samples')

In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install scikit-image==0.16.2



Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Successfully uninstalled keras-2.7.0
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 92.6 MB 1.2 MB/s 
     |████████████████████████████████| 3.2 MB 74.4 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 367 kB 78.8 MB/s 
     |████████████████████████████████| 50 kB 9.3 MB/s 
     |████████████████████████████████| 4.1 MB 58.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found exist

In [ ]:
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import json


# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config


%matplotlib inline 

# Directory to save logs and trained model
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
!pip install scikit-image==0.16.2

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 6  # Background + rupchada and elish

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9



# Prepare Model

In [ ]:
############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "Elish")
        self.add_class("object", 2, "Pabda")
        self.add_class("object", 3, "Poma")
        self.add_class("object", 4, "Rupchada")
        self.add_class("object", 5, "Taki")
        self.add_class("object", 6, "Mola")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open('/content/drive/MyDrive/UMask_RCNN/Dataset/train/train.json'))

        if subset == 'val':
          annotations1 = json.load(open('/content/drive/MyDrive/UMask_RCNN/Dataset/val/val.json'))

        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Elish": 1,"Pabda": 2, "Poma": 3,"Rupchada": 4, "Taki": 5,"Mola": 6}

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )


    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        #print("++++++++++");
        #print(info["height"]);
        
        #print(info["width"]);
        #print("++++++++++");
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


	


In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    #print(dataset_train);
    dataset_train.load_custom("/content/drive/MyDrive/UMask_RCNN/Dataset","train")
    #dataset_train = os.path.join(ROOT_DIR, "/Dataset/train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    #dataset_train = os.path.join(ROOT_DIR, "/Dataset/val")
    dataset_val.load_custom("/content/drive/MyDrive/UMask_RCNN/Dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=2,
                layers='heads')
            
                
                
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)            

objects: ['Poma']
numids [3]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]
objects: ['Poma']
numids [3]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
10/10 [==============================] - 3475s - loss: 3.2810 - rpn_class_loss: 0.0044 - rpn_bbox_loss: 0.3101 - mrcnn_class_loss: 0.8957 - mrcnn_bbox_loss: 1.1839 - mrcnn_mask_loss: 0.8870 - val_loss: 2.1959 - val_rpn_class_loss: 0.0033 - val_rpn_bbox_loss: 0.2047 - val_mrcnn_class_loss: 0.1379 - val_mrcnn_bbox_loss: 1.0233 - val_mrcnn_mask_loss: 0.8267
Epoch 2/2
10/10 [==============================] - 3355s - loss: 2.2302 - rpn_class_loss: 0.0041 - rpn_bbox_loss: 0.2273 - mrcnn_class_loss: 0.1454 - mrcnn_bbox_loss: 1.0517 - mrcnn_mask_loss: 0.8018 - val_loss: 1.6939 - val_rpn_class_loss: 0.0029 - val_rpn_bbox_loss: 0.1933 - val_mrcnn_class_loss: 0.1461 - val_mrcnn_bbox_loss: 0.8518 - val_mrcnn_mask_loss: 0.4999


#TEST#


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

#import custom

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


WEIGHTS_PATH = "/content/drive/MyDrive/Mask_RCNN/logs/object20220116T1842/mask_rcnn_object_0001.h5"   # change it


In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 6  # Background + Rupchada and Elish

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.8

In [ ]:
# Code for Customdataset class. Same code is present in custom.py file also
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "Elish")
        self.add_class("object", 2, "Pabda")
        self.add_class("object", 3, "Poma")
        self.add_class("object", 4, "Rupchada")
        self.add_class("object", 5, "Taki")
        self.add_class("object", 6, "Mola")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open('/content/drive/MyDrive/UMask_RCNN/Dataset/train/train.json'))

        if subset == 'val':
          annotations1 = json.load(open('/content/drive/MyDrive/UMask_RCNN/Dataset/val/val.json'))

        annotations = list(annotations1.values())  # don't need the dict keys

        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Elish": 1,"tab": 2,"Pabda": 3,"tab": 4,"Poma": 5,"tab": 6,"Rupchada": 7,"tab": 8,"Taki": 9,"tab": 10,"Mola": 11}
            #name_dict = {"Rupchada": 1,"Elish": 2}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
ROOT_DIR = "/content/drive/MyDrive/UMask_RCNN/Dataset"

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax



# Load validation dataset
# Must call before using the dataset
CUSTOM_DIR = "/content/drive/MyDrive/UMask_RCNN/Dataset"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

objects: ['Poma']
numids [5]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Elish']
numids [1]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Poma']
numids [5]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Rupchada']
numids [7]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
numids [3]
objects: ['Pabda']
n

In [ ]:
config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load COCO weights Or, load the last model you trained
weights_path = "/content/drive/MyDrive/Mask_RCNN/logs/object20220116T1842/mask_rcnn_object_0001.h5"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
print(model)

Loading weights  /content/drive/MyDrive/Mask_RCNN/logs/object20220116T1842/mask_rcnn_object_0001.h5
Re-starting from epoch 1


In [ ]:

#print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

currentDataClass = [ 3, 1, 1, 1, 1 , 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 3, 3, 4, 4, 4, 4, 4, 2, 5, 5, 5, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]   
predictedClass = []
total_accuracy = []

for i in range(75,80):
  path_to_new_image = '/content/drive/MyDrive/UMask_RCNN/Dataset/test/b ('+str(i)+').jpg'
  image1 = mpimg.imread(path_to_new_image)
  results1 = model.detect([image1], verbose=1)
  r1 = results1[0]
  # ax = get_ax(1)
  single_accuracy = 0
  index = 0
  cur_class = -1
  for r in r1['scores']:
    if r > single_accuracy:
      single_accuracy = r
      cur_class = index
    index+=1
  if cur_class == -1:
    predictedClass.append(0)
  else:
    predictedClass.append(r1['class_ids'][cur_class])
    total_accuracy.append(single_accuracy)
  # visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
  #                             dataset.class_names, r1['scores'], ax=ax, title="Predictions1")

  # print(index)
  # print(single_accuracy)
  # print(r1['class_ids'][cur_class])
  # print(r1['scores']);
  # print(r1['class_ids']);
 


# This is for predicting images which are not present in dataset
#path_to_new_image = '/content/drive/MyDrive/Mask_RCNN/Dataset/IMG_20201204_115936.jpg'
#image1 = mpimg.imread(path_to_new_image)

# Run object detection
#print(len([image1]))
#results1 = model.detect([image1], verbose=1)

# Display results
#ax = get_ax(1)
#r1 = results1[0]
#print(r1['scores']);
#print(r1['class_ids']);
#visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
#dataset.class_names, r1['scores'], ax=ax, title="Predictions1")


Processing 1 images
image                    shape: (600, 800, 3)         min:    6.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 19)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (600, 450, 3)         min:    9.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 19)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (600, 800, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

##Average Accuracy##

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

classes = int(max(currentDataClass) - min(currentDataClass)) + 1 #find number of classes

array = [[sum([(currentDataClass[i] == true_class) and (predictedClass[i] == pred_class) 
                for i in range(len(currentDataClass))])
           for pred_class in range(1, classes + 1)] 
           for true_class in range(1, classes + 1)]

# df_cm = pd.DataFrame(array, range(6), range(6))
# plt.figure(figsize=(10,7))
# sn.set(font_scale=1.4) # for label size
# sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
print("Accuracy",sum(total_accuracy)/len(total_accuracy));
#plt.show()


Accuracy 0.8642759687370725
